## README: This is the final notebook. Use this notebook to paste the final code.
Do not test code here :)

### Pre Processing of Tweets

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import datetime
from dateutil import parser
import re
import emoji
import string

In [145]:
tweets = pd.read_csv('raw_trump_tweets.csv')

In [146]:
# >= 20th jan 2017, <= 31st Dec 2018
tweets['date']= pd.to_datetime(tweets['date'])
tweets = tweets[tweets['date'] >= np.datetime64('2017-01-20')]
tweets = tweets[tweets['date'] <= np.datetime64('2018-12-31')]


In [147]:
def process_and_remove_duplicates(tweets, date):
    duplicated_rows = tweets[tweets['date'] == date]
    id = max(duplicated_rows['id'])
    max_favorite = max(duplicated_rows['favorites'])
    max_retweet = max(duplicated_rows['retweets'])
    combined_tweet = ""
    device = duplicated_rows['device'].iloc[0]
    for index, row in duplicated_rows.iterrows():
        combined_tweet = row['text'] + ' ' + combined_tweet.strip('.')
    tweets = tweets[tweets.date != date]
    row = {'id': id, 'text': combined_tweet, 'isRetweet': 'f', 'isDeleted': 'f', 'device': device, 'favorites': max_favorite, 'retweets': max_retweet, 'date': date}
    tweets = tweets.append(row, ignore_index = True)   
    print(f"Combined {len(duplicated_rows)} tweets on {date}")
    return tweets

In [148]:
dates = tweets["date"]
duplicated = tweets[dates.isin(dates[dates.duplicated()])]
duplicated_dates = duplicated['date'].unique()

for date in duplicated_dates:
    tweets = process_and_remove_duplicates(tweets, date)

Combined 2 tweets on 2018-11-21T22:18:00.000000000
Combined 2 tweets on 2018-09-10T21:59:00.000000000
Combined 2 tweets on 2018-08-14T01:37:00.000000000
Combined 2 tweets on 2018-05-31T10:56:00.000000000
Combined 2 tweets on 2018-02-01T22:37:00.000000000
Combined 2 tweets on 2018-03-09T17:50:00.000000000
Combined 2 tweets on 2017-12-29T12:50:00.000000000
Combined 2 tweets on 2017-12-29T12:48:00.000000000
Combined 3 tweets on 2017-12-28T22:17:00.000000000
Combined 2 tweets on 2017-12-22T20:47:00.000000000
Combined 2 tweets on 2017-12-16T03:09:00.000000000
Combined 2 tweets on 2017-11-18T13:49:00.000000000
Combined 2 tweets on 2017-11-15T10:22:00.000000000
Combined 2 tweets on 2017-07-11T10:53:00.000000000
Combined 2 tweets on 2017-07-11T10:37:00.000000000
Combined 2 tweets on 2017-10-28T21:09:00.000000000
Combined 2 tweets on 2017-10-26T01:47:00.000000000
Combined 2 tweets on 2017-10-14T11:08:00.000000000
Combined 2 tweets on 2017-05-10T10:31:00.000000000
Combined 2 tweets on 2017-09-30

Combined 2 tweets on 2018-06-25T16:58:00.000000000
Combined 2 tweets on 2018-06-23T14:00:00.000000000
Combined 2 tweets on 2018-06-23T11:21:00.000000000
Combined 3 tweets on 2018-06-23T11:19:00.000000000
Combined 2 tweets on 2018-06-23T11:16:00.000000000
Combined 2 tweets on 2018-06-21T20:46:00.000000000
Combined 4 tweets on 2018-06-19T13:52:00.000000000
Combined 2 tweets on 2018-06-18T13:50:00.000000000
Combined 3 tweets on 2018-06-17T14:36:00.000000000
Combined 2 tweets on 2018-06-15T11:56:00.000000000
Combined 4 tweets on 2018-06-15T11:55:00.000000000
Combined 2 tweets on 2018-12-06T08:53:00.000000000
Combined 2 tweets on 2018-06-14T15:09:00.000000000
Combined 2 tweets on 2018-06-14T15:08:00.000000000
Combined 2 tweets on 2018-06-13T09:40:00.000000000
Combined 2 tweets on 2018-12-06T20:40:00.000000000
Combined 2 tweets on 2018-09-06T20:58:00.000000000
Combined 2 tweets on 2018-09-06T20:56:00.000000000
Combined 2 tweets on 2018-06-06T00:40:00.000000000
Combined 2 tweets on 2018-05-30

In [149]:
# tweets that are Trump's 
# remove retweets
tweets1 = tweets[tweets['isRetweet'] == 'f'].reset_index(drop=True)
tweets1 = tweets1[tweets1['text'].str.contains('RT @') == False]
tweets1['cleaned_text'] = tweets1['text'].apply(lambda x: re.sub(r'https?:\/\/\S*', '', x, flags=re.MULTILINE))
tweets1 = tweets1[tweets1['cleaned_text'].str.strip() != '']

# Feauture extraction
tweets2 = tweets1.loc[:, ['id', 'cleaned_text', 'favorites', 'retweets', 'date']]
tweets2['date_new'] = [i + datetime.timedelta(hours = 8) for i in tweets2['date']]
tweets2['date_part'] = [i.date() for i in tweets2['date_new']]
tweets2['time_part'] = [i.time() for i in tweets2['date_new']]
tweets2['hour'] = [int(str(i).split(":")[0]) for i in tweets2['time_part']]
tweets2['year'] = [int(str(i).split("-")[0]) for i in tweets2['date_part']]
tweets2['month'] = [int(str(i).split("-")[1]) for i in tweets2['date_part']]

In [150]:
# Tweet cleaning functions
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_mentions(string):
    return re.sub("@[A-Za-z0-9]+","",string)

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

def clean_tweet(x):
    
    x = remove_emoji(x)
    x = remove_mentions(x)
    x = url_free_text(x)
    # remove punctuations 
    x = re.sub('[%s]' % re.escape(string.punctuation), '', x)
    # remove numbers
    x = re.sub('\w*\d\w*', '', x)
    # remove single quotes
    x = x.strip("'")
    # remove hashtags
    x = x.replace("#", "")
    # remove double quotes
    x = x.strip('"') 
    x = x.lower()
    return x
    
tweets2['cleaned_text_2'] = tweets2.cleaned_text.apply(clean_tweet)

In [151]:
# Remove tweets outside trading hours 
trump_tweets = tweets2.copy()
trump_tweets['date_new'] = pd.to_datetime(trump_tweets['date_new'])
trump_tweets.rename(columns={"date_new": "tweet_datetime"}, inplace=True)
trump_tweets.set_index('tweet_datetime', inplace=True, drop=False)
trump_tweets.index.name = None
trump_tweets = trump_tweets.between_time(datetime.time(4), datetime.time(20), include_start=True, include_end=True)

In [152]:
trump_tweets.to_csv("trump_tweets_cleaned_2.csv", index=False, header=True)

### Pre Processing of Stock Data

In [120]:
# Datasets are combined after pre processing! (else kernel crashes)
stock_price_2017 = pd.read_csv('yr2017_cleaned.csv')
stock_price_2018 = pd.read_csv('yr2018_cleaned.csv')

,DATE,TIME_M,PRICE
0,20170120,04:00:00,226.50
1,20170120,04:04:00,226.49
2,20170120,04:05:00,226.52
3,20170120,04:08:00,226.54
4,20170120,04:10:00,226.52
...,...,...,...
185597,20180119,19:56:00,280.50
185598,20180119,19:57:00,280.55
185599,20180119,19:58:00,280.55
185600,20180119,19:59:00,280.55


In [ ]:
# Drop unwanted columns 
stock_price_2017.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)
stock_price_2018.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)

In [ ]:
# Change to datetime objects
stock_price_2017['TIME_M'] = pd.to_datetime(stock_price_2017['TIME_M'], format='%H:%M:%S.%f')
stock_price_2018['TIME_M'] = pd.to_datetime(stock_price_2018['TIME_M'], format='%H:%M:%S.%f')

In [ ]:
# Remove seconds, milliseconds from data
stock_price_2017['TIME_M'] = stock_price_2017['TIME_M'].dt.floor('Min').dt.time
stock_price_2018['TIME_M'] = stock_price_2018['TIME_M'].dt.floor('Min').dt.time

In [ ]:
# Get minute to minute data (one entry for each minute)
stock_price_2017.drop_duplicates(subset=['TIME_M','DATE'],inplace=True)
stock_price_2018.drop_duplicates(subset=['TIME_M', 'DATE'],inplace=True)

In [122]:
# Export to csv, will be used for preprocessing later!
stock_price = pd.concat([stock_price_2017,stock_price_2018], ignore_index=True, sort=False)
stock_price.to_csv("stock_price.csv", index=False, header=True)
stock_price = pd.read_csv("stock_price.csv", parse_dates=[['DATE', 'TIME_M']])

In [123]:
stock_price

,DATE_TIME_M,PRICE
0,2017-01-20 04:00:00,226.50
1,2017-01-20 04:04:00,226.49
2,2017-01-20 04:05:00,226.52
3,2017-01-20 04:08:00,226.54
4,2017-01-20 04:10:00,226.52
...,...,...
381308,2018-12-31 19:56:00,250.06
381309,2018-12-31 19:57:00,250.07
381310,2018-12-31 19:58:00,250.10
381311,2018-12-31 19:59:00,250.07


In [ ]:
## Interpolation of missing values
idx = pd.date_range(stock_price.iloc[0, 0], stock_price.iloc[-1, 0], freq='1min')
stock_price.index = pd.DatetimeIndex(stock_price.loc[:, 'DATE_TIME_M'])
stock_price = stock_price.reindex(idx, fill_value='NaN')
stock_price['DATE_TIME_M'] = stock_price.index

In [ ]:
stock_price = stock_price.astype({'PRICE': 'float'})
stock_price['PRICE'].interpolate(method='linear', inplace=True)

In [124]:
stock_price.to_csv("stock_price_cleaned.csv", index=False, header=True)

### Combining Stock Data & Tweets

In [156]:
trump_tweets = pd.read_csv("trump_tweets_cleaned_2.csv")
stock_price = pd.read_csv("stock_price_cleaned.csv")

In [157]:
trump_tweets.columns

Index(['id', 'cleaned_text', 'favorites', 'retweets', 'date', 'tweet_datetime',
       'date_part', 'time_part', 'hour', 'year', 'month', 'cleaned_text_2'],
      dtype='object')

In [158]:
stock_price.rename(columns={"DATE_TIME_M": "datetime", "PRICE": "price"}, inplace=True)
stock_price['datetime']= pd.to_datetime(stock_price['datetime'])

#### Get Stock Price Difference X mins after

In [159]:
trump_tweets

,id,cleaned_text,favorites,retweets,date,tweet_datetime,date_part,time_part,hour,year,month,cleaned_text_2
0,9.353400e+17,Thank you Rand!,42793,9125,2017-11-28 02:50:00,2017-11-28 10:50:00,2017-11-28,10:50:00,10,2017,11,thank you rand
1,8.997980e+17,"Join me live from Fort Myer in Arlington, Virg...",36009,4891,2017-08-22 01:00:00,2017-08-22 09:00:00,2017-08-22,09:00:00,9,2017,8,join me live from fort myer in arlington virgi...
2,8.939700e+17,Thank you Nicole!,43367,8275,2017-05-08 23:01:00,2017-05-09 07:01:00,2017-05-09,07:01:00,7,2017,5,thank you nicole
3,8.819770e+17,Thank you to Shawn Steel for the nice words on...,50956,7465,2017-03-07 20:44:00,2017-03-08 04:44:00,2017-03-08,04:44:00,4,2017,3,thank you to shawn steel for the nice words on
4,8.778460e+17,Great night in Iowa - special people. Thank you!,56446,8039,2017-06-22 11:11:00,2017-06-22 19:11:00,2017-06-22,19:11:00,19,2017,6,great night in iowa special people thank you
...,...,...,...,...,...,...,...,...,...,...,...,...
2721,9.990960e+17,If the person placed very early into my campai...,78529,20098,2018-05-23 01:13:00,2018-05-23 09:13:00,2018-05-23,09:13:00,9,2018,5,if the person placed very early into my campai...
2722,9.874600e+17,So General Michael Flynn’s life can be totally...,93569,25259,2018-04-20 10:34:00,2018-04-20 18:34:00,2018-04-20,18:34:00,18,2018,4,so general michael flynn’s life can be totally...
2723,9.870960e+17,"My thoughts, prayers and condolences are with ...",62645,16081,2018-04-19 22:30:00,2018-04-20 06:30:00,2018-04-20,06:30:00,6,2018,4,my thoughts prayers and condolences are with t...
2724,9.863570e+17,Today’s Court decision means that Congress mus...,56749,12426,2018-04-17 21:34:00,2018-04-18 05:34:00,2018-04-18,05:34:00,5,2018,4,today’s court decision means that congress mus...


In [160]:
stock_price

,datetime,price
0,2017-01-20 04:00:00,226.50
1,2017-01-20 04:04:00,226.49
2,2017-01-20 04:05:00,226.52
3,2017-01-20 04:08:00,226.54
4,2017-01-20 04:10:00,226.52
...,...,...
381308,2018-12-31 19:56:00,250.06
381309,2018-12-31 19:57:00,250.07
381310,2018-12-31 19:58:00,250.10
381311,2018-12-31 19:59:00,250.07


In [171]:
def get_stock_price_x_mins_after(trump_tweets, stock_price, x_mins):
    stock_price[f"datetime_{x_mins}mins_before"] = stock_price["datetime"] - pd.Timedelta(f"{x_mins} min")
    stock_price_merged = stock_price.merge(stock_price, how='inner', left_on=[f"datetime_{x_mins}mins_before"], right_on=["datetime"])
    stock_price_merged.drop(columns=["datetime_y", f"datetime_{x_mins}mins_before_y"], inplace=True)
    stock_price_merged.rename(columns={"datetime_x": f"datetime_{x_mins}mins_after", "price_x": f"price_{x_mins}mins_after", f"datetime_{x_mins}mins_before_x": "datetime_now", "price_y": "price_now"}, inplace=True)

    stock_price_merged[f'{x_mins}mins_price_diff_abs'] = stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']
    stock_price_merged[f'{x_mins}mins_price_diff_perc'] = (stock_price_merged[f'price_{x_mins}mins_after'] - stock_price_merged['price_now']) / stock_price_merged['price_now']

    trump_tweets.tweet_datetime = pd.to_datetime(trump_tweets.tweet_datetime)
    stock_price_merged.datetime_now = pd.to_datetime(stock_price_merged.datetime_now)

    merged = trump_tweets.merge(stock_price_merged, how='inner', left_on=['tweet_datetime'], right_on=["datetime_now"])
    # merged.drop(columns=["datetime_now", "price_now", "cleaned_text"], inplace=True)
    # merged.insert(1, 'cleaned_text', merged['cleaned_text2'])
    # merged.drop(columns=["cleaned_text2"], inplace=True)

    return merged

merged = get_stock_price_x_mins_after(trump_tweets, stock_price, 5)

### Get Array of Stock Prices 30 minutes before, ie. [-5, -4, -3, -2, -1]

In [174]:
def get_array_of_prices_xmin_before(datetime_now, x_min, stock_price_merged):
    earliest_time = (datetime_now - timedelta(minutes = x_min)).time()
    if earliest_time >= datetime.time(4, 0):
        result = []
        for min_interval in range(x_min, 0, -1):
            datetime_query = datetime_now - timedelta(minutes = min_interval)
            price = stock_price_merged[stock_price_merged['datetime_now'] == datetime_query].iloc[0, stock_price_merged.columns.get_loc('price_now')]
            result.append(price)
        print(f"{datetime_now}: {result}")
        return result
        
    return None

In [175]:
merged[f'prev_30mins_prices'] = merged['tweet_datetime'].apply(get_array_of_prices_xmin_before, args=(30, stock_price_merged))
merged.to_csv(f"tweets_stocks_combined_final.csv", index=False, header=True)

2017-11-28 10:50:00: [260.96, 261.03, 261.01, 261.015, 261.04, 261.01, 261.02, 261.01, 260.955, 260.94, 260.94, 260.97, 260.97900000000004, 261.01, 261.015, 261.03, 261.08, 261.08, 261.075, 261.125, 261.13, 261.17, 261.105, 261.0861, 261.09, 261.15, 261.11, 261.1, 261.09, 261.0825]


IndexError: single positional indexer is out-of-bounds